In [ ]:
%load_ext sql
%sql sqlite:///store.db

### Про автоинкрементацию

In [ ]:
%%sql
DROP TABLE IF EXISTS Numbers;

CREATE TABLE Numbers(
    id integer PRIMARY KEY,
    val integer NOT NULL
);

INSERT INTO Numbers values (1,1), (2,2), (NULL,3);

SELECT * FROM Numbers;

In [ ]:
%%sql
INSERT INTO Numbers values (10,4), (NULL,5);

SELECT * FROM Numbers;

In [ ]:
2**63-1

In [ ]:
%%sql
INSERT INTO Numbers values (9223372036854775806,6), (NULL,7), (NULL,8), (NULL,9);
SELECT * FROM Numbers;

### SUBSTRING

In [ ]:
%%sql
SELECT * FROM Invoices
LIMIT 5;

In [ ]:
%%sql
SELECT SUBSTRING(InvoiceDate, 12) as time FROM Invoices
LIMIT 5;

In [ ]:
%%sql
SELECT SUBSTRING(InvoiceDate, 1, 10) as date FROM Invoices
LIMIT 5;

In [ ]:
%%sql
SELECT * FROM Invoices
WHERE SUBSTRING(BillingPostalCode, 1, 2) = '70';

In [ ]:
%%sql
SELECT BillingPostalCode, LENGTH(BillingPostalCode) as Codelength
FROM Invoices
LIMIT 5;

In [ ]:
%%sql
SELECT BillingPostalCode, REPLACE(BillingPostalCode, " ", '-') as PostalCode -- не меняет данные таблицы
FROM Invoices
LIMIT 5;

## Работа с TIMEDATE

In [ ]:
%%sql
SELECT * FROM Invoices LIMIT 5;

In [ ]:
%%sql
SELECT * FROM Invoices
WHERE InvoiceDate < '2023-01-08 12:00:00';

In [ ]:
%%sql
SELECT * FROM Invoices
WHERE InvoiceDate BETWEEN '2023-07-08' AND '2023-07-16';

In [ ]:
%%sql
SELECT date(InvoiceDate), time(InvoiceDate) FROM Invoices
WHERE InvoiceDate BETWEEN '2023-07-08' AND '2023-07-16';

### strftime()

Основные форматы

%d		day of month: 01-31

%m		month: 01-12

%Y		year: 0000-9999

%H		hour: 00-24

%M		minute: 00-59

%S		seconds: 00-59

%w		day of week 0-6 with Sunday==0

%j		day of year: 001-366

%W		week of year (00-53) - week 01 starts on the first Monday

https://www.sqlite.org/lang_datefunc.html

In [ ]:
%%sql
SELECT InvoiceDate, strftime('  %d day, %m month  ', InvoiceDate) as date, strftime('  %Hh %Mm', InvoiceDate) as time, strftime('  %w', InvoiceDate) as week FROM Invoices

LIMIT 5;

In [ ]:
%%sql
SELECT datetime('now');

julianday() function returns the Julian day - the fractional number of days since November 24, 4714 B.C.

In [ ]:
%%sql
SELECT julianday(datetime('now'));

In [ ]:
%%sql
SELECT floor(julianday(datetime('now')) - julianday(datetime('2023-09-01'))) as numdays;

### Модификаторы

In [ ]:
%%sql
SELECT datetime('now', 'localtime');

In [ ]:
%%sql
SELECT datetime('now', '+3 hours');

In [ ]:
%%sql
SELECT * FROM Invoices
WHERE date(InvoiceDate) = date('now', '-1 years')
LIMIT 5;

In [ ]:
%%sql
SELECT datetime('now','start of day');

In [ ]:
%%sql
SELECT date('now','start of month','+1 month','-1 day');

In [ ]:
%%sql

--Compute the date of the first Tuesday in October for the current year.

SELECT date('now','start of year','+9 months','weekday 2');

In [ ]:
%%sql
SELECT * FROM Invoices
WHERE date(invoicedate) = date('now','-18 months');

In [ ]:
%%sql
SELECT * FROM Invoices
WHERE date(invoicedate) BETWEEN '2024-02-01' and '2024-03-01';

In [ ]:
%%sql
SELECT * FROM Invoices
WHERE date(invoicedate, 'start of month') = date('2024-02-01');

## Вывести клиентов из одного города

In [ ]:
%%sql
SELECT * FROM Customers LIMIT 10;

In [ ]:
%%sql
SELECT City FROM Customers
GROUP BY City HAVING count(*)>1; -- Вывести города, в которых есть несколько клиентов

In [ ]:
%%sql
SELECT FirstName, LastName, City--, Country
    FROM Customers
    WHERE City IN (SELECT City FROM Customers
                  GROUP BY City HAVING count(*)>1);

In [ ]:
%%sql
SELECT City, Country FROM Customers
                  GROUP BY City, Country HAVING count(*)>1;

In [ ]:
%%sql

WITH RepeatCity as (SELECT City, Country FROM Customers
                    GROUP BY City, Country HAVING count(*)>1)

SELECT FirstName, LastName, City, Country
    FROM Customers
    JOIN RepeatCity using(City, Country);

## Посчитать медиану цен треков

In [ ]:
%%sql

SELECT * FROM tracks LIMIT 5;

### Сначала на пальцах

In [ ]:
%%sql
DROP TABLE IF EXISTS Numbers;

CREATE TABLE Numbers(
    id integer PRIMARY KEY,
    val integer NOT NULL
);

INSERT INTO Numbers(val) values (6),(3),(2),(7),(1),(3),(5);

In [ ]:
%%sql
SELECT val FROM Numbers ORDER BY val;

In [ ]:
%%sql
SELECT avg(val) FROM Numbers;

In [ ]:
%%sql
SELECT val
FROM Numbers
ORDER BY val
LIMIT 1
OFFSET (SELECT COUNT(*)
        FROM Numbers) / 2

In [ ]:
%%sql
INSERT INTO Numbers(val) values (8);
SELECT val FROM Numbers ORDER BY val;

In [ ]:
%%sql
SELECT AVG(val)
FROM (SELECT val
      FROM Numbers
      ORDER BY val
      LIMIT 2
      OFFSET (SELECT (COUNT(*) - 1) / 2
              FROM Numbers))

Всё вместе

In [ ]:
%%sql

WITH cnt as (SELECT COUNT(*) FROM Numbers)

SELECT CASE
    WHEN (SELECT * FROM cnt) % 2 = 0
    THEN
        (SELECT AVG(val) FROM
                (SELECT val FROM Numbers
                    ORDER BY val
                    LIMIT 2
                    OFFSET ((SELECT * FROM cnt) - 1) / 2))
    ELSE
        (SELECT val FROM Numbers
        ORDER BY val
        LIMIT 1
        OFFSET (SELECT * FROM cnt) / 2)
END as median;

или

In [ ]:
%%sql
SELECT AVG(val) as median
FROM (SELECT val
      FROM Numbers
      ORDER BY val
      LIMIT 2 - (SELECT COUNT(*) FROM Numbers) % 2    -- проверка четности
      OFFSET (SELECT (COUNT(*) - 1) / 2
              FROM Numbers))

### Теперь для нашей задачи

In [ ]:
%%sql
SELECT AVG(UnitPrice) as median
FROM (SELECT UnitPrice
      FROM tracks
      ORDER BY UnitPrice
      LIMIT 2 - (SELECT COUNT(*) FROM tracks) % 2
      OFFSET (SELECT (COUNT(*) - 1) / 2
              FROM tracks))